# Q-AnT Manual

Quantitative Analysis Trading (Q-AnT) is a platform to gather and manage information about corporate shares.

This document explains the most important steps in the usage of Q-Ant. It includes the basic commands to perform operation, run the classification algorithm and download input data.

## Initialize

In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import datetime as dt
import time as tt
from QAnT import stock
from QAnT import Index
import datetime

from scipy.stats import norm
from scipy.stats import levy
import matplotlib.mlab as mlab

# Basic functions of the stock and index classes

## Initialization
The initialization of the stock and index classes is done by the following commands:

In [2]:
s = stock(debug=True, verbose=True)

In [3]:
i = Index(debug=True, verbose=True)

reading stored quotes


## Search the stock database
We can search the database by the name of the company.

In [4]:
s = stock(verbose=True)
search_results = s.find_by_name("PROCTER")
search_results

,name,isin,ticker_YF,ticker_MS,branch,benchmark,finanzen_net
2037,PROCTER GAMBLE,US7427181091,PRG,PG,0.0,^GSPC,ProcterGamble


# Downloading data
## Download keyratios from MorningStar and save to database

If we have new companies in our database, we may want to download the keyratios from morningstar.

In [5]:
s.switch_isin('US7427181091')
s.ticker_ms = 'PRG'
s.load_keyratios(deletecsv=False)

File existing: tmpfiles/US_PG.csv


## Download quotes from Yahoo

Let's select the Munich Re stock and download the quote. There are two different modes of operation for the quote downloading algorithm. If the algorithm is called for the first time, Q-AnT tries all exchanges and selects that with the largest available data set. If quotes are already stored for the stock, Q-AnT uses the same exchange as for the previous data. 

In [6]:
s.switch_isin('US7427181091')

In [7]:
s._download_quote_yahoo(useexchange='EUR')



fail

succes
{'':              Open   High    Low  Close  Adj Close     Volume
Date                                                        
2000-01-04  2.200  2.200  2.200  2.200      2.200     9000.0
2000-01-05  2.225  2.400  2.225  2.225      2.225        0.0
2000-01-06  2.000  2.400  2.000  2.000      2.000        0.0
2000-01-07  2.000  2.100  2.000  2.075      2.075    50000.0
2000-01-10  2.050  2.150  2.050  2.050      2.050        0.0
2000-01-11  2.050  2.150  2.050  2.050      2.050        0.0
2000-01-12  2.075  2.075  2.075  2.075      2.075    10000.0
2000-01-13  2.075  2.075  2.075  2.075      2.075    50000.0
2000-01-14  2.075  2.075  2.075  2.075      2.075    70000.0
2000-01-18  2.050  2.075  2.050  2.050      2.050  1024000.0
2000-01-19  2.050  2.150  2.050  2.050      2.050        0.0
2000-01-20  2.075  2.075  2.075  2.075      2.075     5000.0
2000-01-21  2.050  2.150  2.050  2.050      2.050        0.0
2000-01-24  2.075  2.125  2.075  2.075      2.075        0.0
2000

## Download Quarterly Report Dates 

In [8]:
s.debug=False
s._save_unsaved_quarterly_report_dates()

# Deploying the algorithm

## Quantitative Analysis of Fundamentals and Quotes

Run the Q-AnT algorithm for the stock presently loaded. 

In [9]:
s.get_summary(save=True)

Show the result of the quantitative analysis

In [10]:
s.quant_result

,Name,ISIN,Parameter,Value,Point
0,PROCTER GAMBLE,US7427181091,OnlyPositiveEarnings,0,1.0
1,PROCTER GAMBLE,US7427181091,AnnualEarningsGrowth,-1.9,-1.0
2,PROCTER GAMBLE,US7427181091,BookValueGrowth,31.5,0.0
3,PROCTER GAMBLE,US7427181091,HistoricROE,10.5,0.0
4,PROCTER GAMBLE,US7427181091,PresentROE,27.3,1.0
5,PROCTER GAMBLE,US7427181091,PresentRoIC,17.7,1.0
6,PROCTER GAMBLE,US7427181091,EquityRatio,45.8,1.0
7,PROCTER GAMBLE,US7427181091,EBTMargin,20.4,1.0
8,PROCTER GAMBLE,US7427181091,DividendGrowth,nan,0.0


In [11]:
s.summary

,Name,ISIN,FairPrice,FairPricePE,Price,Points
0,PROCTER GAMBLE,US7427181091,58.265791,7.947321,3.01,4.0


If we want to save the output to 

    output/algo_results.db
    
we can use the save option. The conservative option is used to specify wheter the calculation of the fair price should take into account earnings growth which is determined from historical earnings growth.

In [12]:
s.get_summary(save=True,conservative=True)
s.analyze_quote()

In [13]:
s.quant_result

,Name,ISIN,Parameter,Value,Point
0,PROCTER GAMBLE,US7427181091,OnlyPositiveEarnings,0,1.0
1,PROCTER GAMBLE,US7427181091,AnnualEarningsGrowth,-1.9,-1.0
2,PROCTER GAMBLE,US7427181091,BookValueGrowth,31.5,0.0
3,PROCTER GAMBLE,US7427181091,HistoricROE,10.5,0.0
4,PROCTER GAMBLE,US7427181091,PresentROE,27.3,1.0
5,PROCTER GAMBLE,US7427181091,PresentRoIC,17.7,1.0
6,PROCTER GAMBLE,US7427181091,EquityRatio,45.8,1.0
7,PROCTER GAMBLE,US7427181091,EBTMargin,20.4,1.0
8,PROCTER GAMBLE,US7427181091,DividendGrowth,nan,0.0
9,PROCTER GAMBLE,US7427181091,OnlyPositiveEarnings,0,1.0


We can have a look at the fair price calculated by means of different valuation models. 

In [14]:
s.price, s.fairprice_low, s.fairprice_high, s.fairprice_pe

(3.0099999999999998,
 60.230619697110782,
 56.300962633181456,
 7.9473214285714286)

# Study daily returns, outperformance, covariance etc.

We can merge two quote dataframes to study covariance or outperformance. 

In [ ]:
from QAnT.risk import merge_quotes

## Correlation

Let's study the covariance first between Coca-Cola and Pepsi and then between Coca-Cola and VISA.
### Between Coca Cola and Pepsi

In [ ]:
s.find_by_name("PEPS")

In [ ]:
cocac = stock(isin='US1912161007')
pepsi = stock(isin='US7134481081')
visa  = stock(isin='US92826C8394')

In [ ]:
plt.plot(cocac.quote['date'],cocac.quote['close'],label='Coca Cola')
plt.plot(pepsi.quote['date'],pepsi.quote['close'],label='Pepsi')
plt.plot(visa.quote['date'],visa.quote['close'],label='VISA')

plt.legend()
plt.show()

In [ ]:
allquotes = merge_quotes(cocac,pepsi, visa)
allquotes.head()

In [ ]:
plt.hist(allquotes['return_COCA-C'].dropna(),bins=100,histtype='step', normed=True)
plt.hist(allquotes['return_PEPSIC'].dropna(),bins=100,histtype='step', normed=True)
plt.hist(allquotes['return_VISA I'].dropna(),bins=100,histtype='step', normed=True)

plt.xlim(-0.1,0.1)
plt.show()

In [ ]:
covmatrix = allquotes.cov()
covmatrix

In [ ]:
allquotes.describe()

In [ ]:
w1 = np.arange(0,1,0.01)
w2 = 1-w1

In [ ]:
cov  = covmatrix[allquotes.keys()[1]][allquotes.keys()[3]]
var1 = allquotes[allquotes.keys()[1]].std()**2
var2 = allquotes[allquotes.keys()[3]].std()**2
r1   = allquotes[allquotes.keys()[1]].mean()
r2   = allquotes[allquotes.keys()[3]].mean()

In [ ]:
def _portfolio_return_var(r1,r2,w1,w2,var1,var2,cov):
    preturn   = w1*r1 + w2*r2
    pvariance = w1*w1*var1 + w2*w2*var2
    pvariance = pvariance + 2*w1*w2*cov
    return preturn, pvariance

def portfolio_return_var(allquotes,col1,col2):
    cov  = covmatrix[allquotes.keys()[col1]][allquotes.keys()[col2]]
    var1 = allquotes[allquotes.keys()[col1]].std()**2
    var2 = allquotes[allquotes.keys()[col2]].std()**2
    r1   = allquotes[allquotes.keys()[col1]].mean()
    r2   = allquotes[allquotes.keys()[col2]].mean()
    return _portfolio_return_var(r1,r2,w1,w2,var1,var2,cov)

In [ ]:
for j in range(1,3):
    output = []
    for w1 in np.arange(0,1,0.01):
        w2 = 1-w1
        preturn, pvariance = portfolio_return_var(allquotes,j,3)
        output.append([w1,preturn,pvariance])
    output = pd.DataFrame(output,columns=['w1','pret','pvar'])

    plt.scatter(np.sqrt(output['pvar']),output['pret']*100)
plt.show()

In [ ]:
plt.scatter(np.sqrt(output['pvar']),output['pret']*100)
plt.show()

## Multi asset portfolios

In [ ]:
def _get_returns_variance(allquotes):
    returns  = []
    variance = []
    for j in allquotes.keys()[1:]:
        returns.append(np.mean(allquotes[j]))
        variance.append(np.std(allquotes[j])**2)

    returns  = np.array(returns)
    variance = np.array(variance)
    return returns, variance
    
def get_portfolio_return_risk(allquotes,weights=None):
    # get the returns and the variances for the individual assets
    returns, variance = _get_returns_variance(allquotes)
    
    nassets = len(allquotes.keys()[1:]) # number of assets in the portfolio

    if weights is None:                     # use random weights unless the user specifies them 
        weights = np.random.rand(nassets)   # 
        weights = weights/np.sum(weights)   # 
    
    # calculate the portfolio return
    portfolio_return = np.sum(returns*weights)
    
    # calculate the covariance matrix
    covmatrix = allquotes.cov()

    # calculate the portfolio variance
    portfolio_variance = np.sum(weights*weights*variance)

    for ii, i in enumerate(covmatrix.keys()):
        for jj, j in enumerate(covmatrix.keys()):
            if i==j:
                continue
            portfolio_variance += covmatrix[i][j]*weights[ii]*weights[jj]
            
    return weights, portfolio_return, portfolio_variance

def calculate_efficient_frontier(allquotes,niter=5000):
    output = []
    allweights = []
    for _ in range(niter):
        weights, pret, pvar = get_portfolio_return_risk(allquotes)
        output.append([pret, pvar])
        allweights.append(weights)
    output = pd.DataFrame(output, columns=['pret','pvar'])
    
    return allweights, output

def get_min_std_portfolio(allquotes,allweights=None, output=None, niter=5000):

    if allweights is None or output is None:
        allweights, output  = calculate_efficient_frontier(allquotes,niter=niter)
    minrisk_index = output['pvar'].idxmin()

    return allweights[minrisk_index], output['pvar'][minrisk_index], output['pret'][minrisk_index]*100

In [ ]:
allquotes = merge_quotes(cocac, visa, pepsi)

In [ ]:
_, output = calculate_efficient_frontier(allquotes)

In [ ]:
plt.scatter(output['pvar']**0.5, output['pret']*100)

for i in allquotes.keys()[1:]:
    plt.plot(allquotes[i].std(),allquotes[i].mean()*100,'r^')

plt.xlabel("Portfolio Standard Deviation on Expected Return")
plt.ylabel("Expected Return [%]")
plt.show()

We can now calculate the portfolio with the lowest risk

In [ ]:
weights, varminrisk, retminrisk = get_min_std_portfolio(allquotes)

In [ ]:
weights

The standard deviation and expected return for this portfolio are

In [ ]:
varminrisk**0.5, retminrisk

## Include more assets

In [ ]:
gilead  = stock(isin='US3755581036')
flavors = stock(isin='US4595061015')
visa    = stock(isin='US92826C8394')
fuchs   = stock(isin='DE0005790430')
johnson = stock(isin='US4781601046')

In [ ]:
allquotes = merge_quotes(gilead, flavors, visa, fuchs, johnson)

In [ ]:
allweights, output = calculate_efficient_frontier(allquotes,niter=10000)

In [ ]:
plt.scatter(output['pvar']**0.5, output['pret']*100)

for i in allquotes.keys()[1:]:
    plt.plot(allquotes[i].std(),allquotes[i].mean()*100,'r^')

plt.xlabel("Portfolio Standard Deviation on Expected Return")
plt.ylabel("Expected Return [%]")
plt.show()

In [ ]:
weights, varminrisk, retminrisk = get_min_std_portfolio(allquotes,allweights=allweights, output=output)

In [ ]:
weights, varminrisk**0.5, retminrisk

### Between Coca Cola and VISA

In [ ]:
s.find_by_name("VISA")

In [ ]:
visa = stock(isin='US92826C8394')
visa._calculate_volatility()
visa.quote['dailyreturn'] = visa.dailychange
visa.dailychange.dropna(inplace=True)

In [ ]:
both = pd.merge(cocac.quote, visa.quote,on='date')
both = both[['date','name_x','name_y','close_x','close_y','dailyreturn_x','dailyreturn_y']]

In [ ]:
both[['dailyreturn_x','dailyreturn_y']].corr()

In [ ]:
visa.quote

# Plotting 

## Interactive Summary
Q-AnT includes a basic plotting algorithm to summarize essential quantities. It can be called using the following function which will load an interactive plot in a new tab.

In [ ]:
s.interactive_summary()

## Basic Plotting

In [ ]:
s.switch_isin('DE0005408116')
plt.plot(s.quote['date'], s.quote['close'],'.')
plt.xlabel("Date")
plt.ylabel("Close [EUR]")
plt.xlim(datetime.date(2010, 4, 1), datetime.date.today())
plt.show()

In [ ]:
plt.plot(i.quote_saved['^GDAXI']['date'], i.quote_saved['^GDAXI']['close'])
plt.xlim(datetime.date(2010, 4, 1), datetime.date.today())

plt.show()

In [ ]:
def merge_quotes(quote1, quote2):
    '''Returns a dataframe with the merged quotes'''
    newdf = pd.merge(quote1, quote2, how='inner',on='date')
    newdf = newdf[['date','name_x','name_y','close_x','close_y']]
    newdf = newdf.assign(return_x=newdf['close_x'].diff()/newdf['close_x'][1:])
    newdf = newdf.assign(return_y=newdf['close_y'].diff()/newdf['close_y'][1:])
    return newdf

In [ ]:
newdf = merge_quotes(i.quote_saved['^GDAXI'], s.quote)
# newdf = newdf.head()
newdf.head()

In [ ]:
newdf[['return_x','return_y']].corr()

In [ ]:
def _assign_colnames(df):
    name = df['name'][0][0:6]
    df   = df[['date','close']]    
    df   = df.assign(dailyreturn=df['close'].diff()/df['close'][1:])
    df.columns = ['date','{0}'.format(name), 'return_{0}'.format(name)]    
    return df

def merge_quotes(*args):
    '''Merge multiple quote dataframes to allow studies of correlation'''
    newdf = _assign_colnames(args[0])
    for k in range(1,len(args)):
        dftoadd = _assign_colnames(args[k])
        newdf = pd.merge(newdf, dftoadd, how='inner',on='date')
    return newdf

In [ ]:
df = merge_quotes(i.quote_saved['^GDAXI'], s.quote, t.quote)

In [ ]:
df.corr()

In [ ]:
plt.hist((df['return_AAREAL']-df['return_DAX'])[1:],bins=100)
plt.show()

In [ ]:
plt.plot(df['return_AAREAL']-df['return_DAX'], df['return_MUENCH']-df['return_DAX'],'o')
plt.xlabel("Outperformance AAREAL")
plt.ylabel("Outperformance MUNICH RE")
plt.show()

In [ ]:
plt.hist(np.diff(df['return_AAREAL']-df['return_DAX'])[1:],bins=100)
plt.show()

In [ ]:
t = stock(verbose=True, isin='DE0008430026')

# Logging

The stock and index classes save every generated output to the logging databases stored in the files

    output/algo.log
    output/algo.err